# TRABALHO 3

Modelação e implementação em SMT de um autómato híbrido que descreva o sistema de travagem ABS e verifique as suas propriedades dinâmicas.
O sistema usa 2 variáveis contínuas para descrever a  `velocidade do veículo` em relação ao solo e a `velocidade linear dos pneus` também em relação ao solo. O sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades. 
Como primeira abordagem a componente discreta do sistema contém os seguintes modos:
`Start`, `Free`, `Stopping`, `Blocked`, e `Stopped`.



## Autómatos híbridos


*Autómatos híbridos* são modelos de sistemas ciber-físicos. Essencialmente um autómato híbrido é uma *máquina de estados finita*, onde cada estado (designado por *modo de funcionamento*) descreve o comportamento contínuo de um sistema dinâmico modelado por *relações diferenciais ordinárias* (nas variáveis contínuas e nas suas derivadas em relação ao tempo) codificadas num predicado designado por *flow*. Estas variáveis contínuas evoluem num modo de funcionamento enquanto o seu *flow* for válido.
Cada transição discreta entre estados é representada por um arco anotado com um predicado (designado *switch* ou *jump*). Uma transição realiza-se quando o seu *switch* é válido. Ao ocorrer uma transição as variáveis internas dos modos conservam o seu valor, a não ser que lhes seja explicitamente atribuído um novo valor.

Um autómato híbrido pode ser descrito por um FOTS (sobre o qual podemos verificar propriedades lógicas com as metodologias que já estudamos). Nesse processo o FOTS vai *discretizar* as relações diferenciais ordinárias e vai mapea-las num espaço de estados também discreto.

### 1.  Defina um autómato híbrido que descreva a dinâmica do sistema de travagem ABS.

INSERIR IMAGEM E CONDIÇOES

### 2. Modele em lógica temporal LT  propriedades que, na sua opinião, caracterizam o comportamento desejável do sistema. Nomeadamente (mas não só”) a propriedade   ”o veículo para em menos de $t$ segundos” e “a velocidade $V$ diminui sempre com o tempo”.

INSERIR IMAGEM E CONDIÇOES

### 3. Codifique em SMT’s o modelo que definiu em 1.

In [2]:
from z3 import *

Comecemos por declarar os modos.
Na codificação em Z3 é conveniente usar um tipo enumerado para implementar os modos:

In [3]:
Mode,(Start, Free, Stopping, Blocked, Stopped) = EnumSort('Mode',('Start', 'Free', 'Stopping', 'Blocked', 'Stopped'))

Podemos agora declarar as variáveis do FOTS correspondente ao sistema ABS da seguinte forma:

In [4]:
def declare(i):
    s ={}
    s['M'] = Const('M'+str(i), Mode)    # constante do tipo mode
    s['V'] = Real('V'+str(i))           # Velocidade do veículo
    s['v'] = Real('v'+str(i))           # velocidade da roda
    s['F'] = Real('F'+str(i))           # Força F
    s['f'] = Real('f'+str(i))           # força f
    s['P'] = Int('P'+str(i))            # peso
    s['a'] = Real('a'+str(i))           # força de atrito
    s['c'] = Real('c'+str(i))           # valor c
    s['T'] = Real('T'+str(i))           # varíavel contínua que denota o tempo
    return s

De seguida iremos codificar os predicados Z3 `init`, `trans` e `inv`, que caracterizam, respectivamente, os estados iniciais, as transições e o invariante de modo do FOTS correspondente ao sistema ABS.

 - Proposição do estado inicial:
$$
T = 0 \wedge M = \mathit{Start} \wedge v = 15 \wedge V = 30 \wedge c = 4 \wedge P = 100 \wedge f = a * P  \wedge a = 0.4 \wedge F = c * (V - v)
$$


 - Proposição do invariante:
$$
V - v \ge 0 \wedge V \ge 0 \wedge v \ge 0
$$ 

In [5]:
def init(s): ## 40 20, 
    return And(s['M'] == Start, s['v'] == 15, s['V'] == 30, s['F'] == s['c'] * (s['V'] - s['v']), \
               s['a'] == 0.4, s['P'] == 100, s['f'] == s['a'] * s['P'], s['c'] == 4, s['T'] == 0)

def inv(s):
    return And(s['V'] - s['v'] >= 0, s['V'] >= 0, s['v'] >= 0)

 - Proposições comuns:
 
$$
\begin{array}{c}
equacao_F = F \ge 0 \wedge F' = c' * (V' - v') \\
equacao_f = f \ge 0 \wedge f = a * P \\
constante_P = P = 100 \wedge P' = P \\
constante_a = a = 0.4 \wedge a' = a \\
equacao_V = V' - V = (-c * (V - v)) * (T' - T) \\
equacao_v = v' - v = (-a * P + c * (V - v)) * (T' - T) \\
propsComuns = equacao_F \wedge equacao_f \wedge constante_P \wedge constante_a \\
erro = 1 \\
\Delta t = 0.1
\end{array}
$$ 
 
 
 
 - Proposição das transições timed:
 
$$
\begin{array}{c}
M = \mathit{Free} \wedge M' = M \wedge V \ge 0 \wedge v \ge 0 \wedge c = 4 \wedge T' = T + \Delta t \wedge c' = c \wedge propsComuns \wedge equacao_v \wedge equacao_V \\
\vee\\
M = \mathit{Stopping} \wedge M' = M \wedge V \ge 0 \wedge v \ge 0 \wedge c = 10 \wedge T' = T + \Delta t \wedge c' = c \wedge propsComuns \wedge equacao_V \wedge equacao_v \\
\vee\\
M = \mathit{Blocked} \wedge M' = M \wedge V = v \wedge c = 4 \wedge T' = T + \Delta t \wedge c' = c \wedge propsComuns \wedge equacao_v \wedge equacao_V
\end{array}
$$



 - Proposição das transições untimed:
 
 $$
\begin{array}{c}
M = \mathit{Start} \wedge M' = \mathit{Free} \wedge V \ge 0 \wedge v \ge 0 \wedge c = 4 \wedge V' = V \wedge v' = v \wedge c' = c \wedge propsComuns \wedge T' = T \\
\vee\\
M = \mathit{Free} \wedge M' = \mathit{Stopping} \wedge v > 0 \wedge V > 1.5 * v \wedge c = 4 \wedge V' = V \wedge v' = v \wedge c' = 10 \wedge propsComuns \wedge T' = T \\
\vee\\
M = \mathit{Stopping} \wedge M' = \mathit{Blocked} \wedge c = 10 \wedge V' = V \wedge v' = v \wedge c' = 4 \wedge propsComuns \wedge T' = T \\
\vee\\
M = \mathit{Stopping} \wedge M' = \mathit{Stopped} \wedge c = 10 \wedge V' = V \wedge v' = v \wedge c' = 0 \wedge propsComuns \wedge T' = T \\
\vee\\
M = \mathit{Blocked} \wedge M' = \mathit{Free} \wedge c = 4 \wedge V' = V \wedge v' = v \wedge c' = c \wedge propsComuns \wedge T' = T
\end{array}
$$

In [6]:
def trans(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0, p['F'] == p['c'] * (p['V'] - p['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    constante_P = And(s['P'] == 100, p['P'] == s['P'])
    constante_a = And(s['a'] == 0.4, p['a'] == s['a'])
    # Equações com tempo (ao adicionar o tempo, deixam de ser comuns e apenas ocorrem nos loops):
    equacao_V = p['V'] - s['V'] == (-s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T'])
    equacao_v = p['v'] - s['v'] == (-s['a'] * s['P'] + s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T'])
    
    # proposições comuns
    props_comuns = And(equacao_F, equacao_f, constante_P, constante_a)
    erro = 1
    fracao_tempo = 0.1
    
    # timed
    infree = And(s['M'] == Free, s['V'] >= 0, s['v'] >= 0, s['c'] == 4, \
                 p['M'] == s['M'], p['c'] == s['c'], \
                 props_comuns, equacao_V, equacao_v, \
                 p['T'] == s['T'] + fracao_tempo)
    
    instopping = And(s['M'] == Stopping, s['V'] >= 0, s['v'] >= 0, s['c'] == 10, \
                     p['M'] == s['M'], p['c'] == s['c'], \
                     props_comuns, equacao_V, equacao_v, \
                     p['T'] == s['T'] + fracao_tempo)
    
    inblocked = And(s['M'] == Blocked, \
                    p['M'] == s['M'], s['V'] == s['v'], s['c'] == 4, \
                    props_comuns, equacao_V, equacao_v, p['c'] == s['c'], \
                    p['T'] == s['T'] + fracao_tempo)
    
    # untimed
    startTofree = And(s['M'] == Start, s['V'] >= 0, s['v'] >= 0, s['c'] == 4, \
                      p['M'] == Free, p['V'] == s['V'], p['v'] >= s['v'], p['c'] == s['c'], \
                      props_comuns, p['T'] == s['T'])
    freeTostopping = And(s['M'] == Free, s['v'] > 0, s['V'] > 1.5 * s['v'], s['c'] == 4, \
                         p['M'] == Stopping, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 10, \
                         props_comuns, \
                         p['T'] == s['T'])
    stoppingToblocked = And(s['M'] == Stopping, s['c'] == 10, \
                            p['M'] == Blocked, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 4, \
                            props_comuns, \
                            p['T'] == s['T'])
    stoppingTostopped = And(s['M'] == Stopping, s['c'] == 10, s['v'] <= 1, \
                            p['M'] == Stopped, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 0, \
                            props_comuns, \
                            p['T'] == s['T'])
    blockedTofree = And(s['M'] == Blocked, s['c'] == 4, \
                        p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                        props_comuns, \
                        p['T'] == s['T'])
    
    return Or(infree, instopping, inblocked, startTofree, freeTostopping, stoppingToblocked, blockedTofree, \
              stoppingTostopped)

In [42]:
def gera_traco(declare,init,trans,inv,k):
    s = Solver()
    
    #criar k cópias do estado, guardar na lista do traço
    trace = []
    for i in range(k):
        trace.append(declare(i))
        
    #restriçoes init e trans    
    s.add(init(trace[0]))
    
    for i in range(k):
        s.add(inv(trace[i]))
    
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i + 1]))

    print(s.check())
    
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
                
gera_traco(declare, init, trans, inv, 50)

sat
0
M = Start
V = 30.0
v = 15.0
F = 60.0
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
1
M = Free
V = 30.0
v = 19.16666676622222
F = 43.333332935111116
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
2
M = Stopping
V = 30.0
v = 19.16666676622222
F = 108.33333233777779
f = 40.0
P = 100
a = 0.4
c = 10.0
T = 0.0
3
M = Blocked
V = 30.0
v = 19.16666676622222
F = 43.333332935111116
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
4
M = Free
V = 30.0
v = 19.16666676622222
F = 43.333332935111116
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
5
M = Stopping
V = 30.0
v = 19.16666676622222
F = 108.33333233777779
f = 40.0
P = 100
a = 0.4
c = 10.0
T = 0.0
6
M = Blocked
V = 30.0
v = 19.16666676622222
F = 43.333332935111116
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
7
M = Free
V = 30.0
v = 19.16666676622222
F = 43.333332935111116
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.0
8
M = Free
V = 25.66666670648889
v = 19.50000005973333
F = 24.666666587022224
f = 40.0
P = 100
a = 0.4
c = 4.0
T = 0.1
9
M = Free
V = 23.200000047786666
v = 1

### 2 
Modele em lógica temporal LT  propriedades que, na sua opinião, caracterizam o comportamento desejável do sistema. Nomeadamente (mas não só) a propriedade ”o veículo para em menos de $t$ segundos” e “a velocidade $V$ diminui sempre com o tempo”.

 - Primeira propriedade:  o veículo para em menos de t segundos.

$$
M = Stopped \rightarrow T < t
$$ 

In [12]:
def stopIn3Seconds(s):
    return Implies(s['M'] == Stopped, s['T'] < 3)

 - Segunda propriedade: A velocidade V diminui sempre com o tempo.

$$
V \le 30
$$ 

In [8]:
def velocidadeDiminui(s):
    return s['V'] <= 30

 - Terceira propriedade: o Veículo chega eventualmente ao estado Stopped.

$$
(v \le 1 \wedge c = 0) \rightarrow M = Stopped
$$ 

In [89]:
def veiculoPara(s):
    return Implies(And(s['v'] <= 1, s['c'] == 0), s['M'] == Stopped)

 ### 4
 Codifique a verificação das propriedades temporais que definiu em 2.

In [10]:
def bmc_always(declare,init,trans,inv,prop,K):
    s = Solver()
    
    # criar k cópias de estado, guardar na lista do traço
    trace = []
    for i in range(K):
        trace.append(declare(i))
        
    s.add(init(trace[0]))
    
    for i in range(K):
        s.add(inv(trace[i]))
    
    for i in range(K - 1):
        s.add(trans(trace[i], trace[i + 1]))
        
    s.add(Not(prop(trace[K - 1])))
    
    if s.check() == sat:
        m = s.model()
        for i in range(K):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
        return
    
    print ("A propriedade é válida de em traços de tamanho até " + str(K))


 - Primeira propriedade:  o veículo para em menos de t segundos.

In [11]:
bmc_always(declare, init, trans, inv, stopIn3Seconds, 20)

A propriedade é válida de em traços de tamanho até 20


 - Segunda propriedade: A velocidade V diminui sempre com o tempo.

In [88]:
bmc_always(declare, init, trans, inv, velocidadeDiminui, 20)

A propriedade é válida de em traços de tamanho até 20


 - Terceira propriedade: o Veículo chega eventualmente ao estado Stopped.

In [90]:
bmc_always(declare, init, trans, inv, veiculoPara, 50)

A propriedade é válida de em traços de tamanho até 50
